<a href="https://colab.research.google.com/github/FabiolaAJ/chatbot-assistente-de-vagas/blob/main/Chatbot_Assistente_de_Vagas_de_emprego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
!pip install -q google-adk

In [ ]:
import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
from IPython.display import clear_output

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Vagas --- #
##########################################
def agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente simpático especializado na busca de vagas de emprego.
        Use a ferramenta de busca do google (google_search) para encontrar até 5 vagas relevantes e atuais para a profissão e senioridade do usuário.

          1. **Interpretação:** Analise profissão e senioridade. Se a senioridade faltar, pergunte.
          2. **Busca (google_search):** Combine profissão, área, senioridade e informe se a vaga é hibrida, presencial ou remoto. Foque em resultados que provavelmente contenham links diretos para a página da vaga.
          3. **Filtragem:** Priorize vagas recentes verificando a data de hoje (data_de_hoje). Descarte vagas antigas ou encerradas.
          4. **Resposta (máximo 5 vagas):** Para cada vaga, retorne título, empresa, localização (se disponível), breve descrição com requisitos para vaga, não faça questionamentos adicionais para o usuário.
          5. **Link:** [**Link direto para a página da vaga**. Este é um campo obrigatório. Tente ao máximo encontrar o link nos resultados da busca.]
          6. **Sem Resultados:** Se não houver vagas relevantes e atuais, informe o usuário.

          Priorize informações precisas e atuais.

        """,
        description="Agente que busca informações de vagas de empregos no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Profissão: {profissao} \n Area:{area} \n Senioridade:{senioridade} \n Data de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de carreira e recrutamento --- #
################################################
def agente_planejador_de_carreira(profissao, area, senioridade, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em carreira e recrutamento muito gentil e simpático, com vasta experiência em ajudar candidatos a se prepararem para processos seletivos.
        Gere dicas para se destacar, considerando os seguintes tópicos:
             1. Quais habilidades e experiências o candidato deve enfatizar?
             2. Forneça até 2 dicas concisas sobre como o usuário pode se organizar para se candidatar as vagas (ex: adaptar currículo, pesquisar a empresa).
             3. Ao final cite dicas gerais de como se destacar no mercado de trabalho de acordo com a profissão do usuário
        """,
        description="Agente planejador especialista em carreira",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Profissão: {profissao} \n Area:{area}  \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados}"

    # Executa o agente
    lancamento_planejamento = call_agent(planejador, entrada_do_agente_planejador)
    return lancamento_planejamento

In [20]:
def agente_responder_duvidas(profissao, area, senioridade, duvida, lancamentos_buscados, lancamento_planejamento):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de carreira e você sempre responde de forma gentil e de facil entendimento.
        Use a ferramenta de busca do google (google_search) para tirar a dúvida do usuário sobre sua carreira, responda de forma resumida.
        """,
        description="Agente para tirar dúvidas de carreira",
        tools=[google_search]
    )

    entrada_do_agente_responder_duvidas = f"Dúvida: {duvida}\n Profissao:{profissao} \n Area:{area} \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados} \n Planejador: {lancamento_planejamento}"

    lancamentos = call_agent(buscador, entrada_do_agente_responder_duvidas)
    return lancamentos

In [ ]:
def obter_input_obrigatorio(mensagem_inicial, mensagem_erro):
    # Exibe a mensagem inicial para o usuário, solicitando a entrada.
    valor = input(f"\n{mensagem_inicial} ")

    # Inicia um loop 'while' que continuará executando enquanto a condição for verdadeira.
    # A condição 'not valor.strip()' verifica se a variável 'valor' está vazia ou contém apenas espaços em branco.
    # 'valor.strip()' remove os espaços em branco do início e do final da string.
    # Se após a remoção dos espaços, a string estiver vazia, a condição será 'True' e o loop continuará.
    while not valor.strip():
        # Exibe a mensagem de erro informando ao usuário que a entrada é obrigatória.
        print(f"{mensagem_erro}")

        # Solicita novamente a entrada do usuário, exibindo a mensagem inicial novamente.
        valor = input(f"\n{mensagem_inicial}")

    # Quando o loop 'while' terminar (ou seja, o usuário digitou algo que não é apenas espaço),
    # a função retorna o valor digitado, com quaisquer espaços em branco iniciais e finais removidos
    # usando o método 'strip()'.
    return valor.strip()

In [ ]:
def tirar_duvidas(profissao, area, senioridade, resposta_buscador, resposta_planejador):
    questionario = "S"

    while questionario.upper() == "S" :
      print("\n Ficou com alguma dúvida? Pode contar comigo!😉 (caso não tenha dúvidas digite 'N') \n")
      duvida = input("")
      if(duvida.upper() == 'N'):
        break;
      else:
        print("\n🔄 Processando resposta...")
        resposta_duvida = agente_responder_duvidas(profissao, area, senioridade, duvida, resposta_buscador, resposta_planejador)
        display(to_markdown(resposta_duvida))

        print("\n Se deseja tirar outra dúvida digite 'S'(caso não tenha, digite qualquer tecla): \n")
        questionario = input("")

        if(questionario.upper() != "S"):
          break;


In [ ]:
def exibir_resultados(profissao, area, senioridade):
    print(f"")

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    resposta_buscador = agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje)
    print("\n 📝 Essas foram as vagas encontradas! \n")
    display(to_markdown(resposta_buscador))
    print("--------------------------------------------------------------")

    print("\n🔄 Processando resultados de planejamento...")
    resposta_planejador = agente_planejador_de_carreira(profissao, area, senioridade, resposta_buscador)
    print("\n 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 \n")
    display(to_markdown(resposta_planejador))
    print("--------------------------------------------------------------")

    tirar_duvidas(profissao, area, senioridade, resposta_buscador, resposta_planejador)

    return

In [19]:
def main():
  exibir_questionario = "1";

  while exibir_questionario == "1" :
    clear_output(wait=True)

    print("\n Olá, tudo bem por aí? 😊 Que bom te ter aqui!")
    print("\n Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉")
    print("\n--------------------------------------------------------------")

    profissao = obter_input_obrigatorio("❓Me conta qual a profissão que te interessa encontrar vagas hoje? ", "/n Hmm, para que eu te ajude a achar as melhores vagas, preciso que me conte qual profissão você busca, tá bem? 😉 ")
    area = obter_input_obrigatorio("❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend) " , "É importante me informar qual área específica dentro dessa profissão deseja atuar!😊 ")
    senioridade = obter_input_obrigatorio("❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...) ", "Porfavor, digite seu nível de experiência para refinar a busca! 😊 ")

    print("\n🔄 Processando resultados...")
    exibir_resultados(profissao, area, senioridade)

    print("\n Se deseja procurar outra vaga, digite 1 (para encerrar digite qualquer tecla): \n")

    exibir_questionario = input("")

    if(exibir_questionario != '1'):
      print("\n Até a próxima! 😊")
      break;


main()




 Olá, tudo bem por aí? 😊 Que bom te ter aqui!

 Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉

--------------------------------------------------------------

❓Me conta qual a profissão que te interessa encontrar vagas hoje?  Programador

❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend)  Full stack

❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...)  Júnior

🔄 Processando resultados...


 📝 Essas foram as vagas encontradas! 



> Com certeza! Aqui estão algumas vagas de Programador Full Stack Júnior, com base na sua solicitação e nas informações mais recentes:
> 
> 1.  **Desenvolvedor Full Stack Jr. - LUZA Group**
> 
>     *   Localização: São Paulo, SP
>     *   Requisitos: Não especificados
>     *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEVEc7gpofN4dum73NRuD5ew6dcOigyLkJTk-MSHovKSrWKQeJt2nFMWALV9a-UqOSxzN9bPUrxt6kDmvqvMjkl8_MB32QX7mqvI5Ft8UFAiaHRLovnbXiyHmCLbmTc7Yu-6-HAPu4rTndRHqaoKA==)
> 2.  **Desenvolvedor Full Stack Jr – P&D - PADO S.A.**
> 
>     *   Localização: Cambé, Paraná
>     *   Requisitos: Não especificados
>     *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGnfo9WrZMXyHRwyZ8xlxfFgVQXO7Gvn_cFRfhEDvkGY8nuYovLDWxb03woEwidLAjjbiqeYZY2khrcCeLZK0L6xAAzQ9M45OO7Lmn51LmKdHbaagG5YaKrHD8owzol3StuVFD1eN-uKJ57OVj6ug==)
> 3.  **Desenvolvedor Full-Stack - Júnior - PHP+React - FIESC (Serviço Nacional de Aprendizagem Industrial)**
> 
>     *   Localização: Florianópolis - SC
>     *   Modelo: Híbrido – SP ou RJ (2x por semana presencial)
>     *   Requisitos: React, Nodejs, Jest, Restify ou similar, API Rest, Docker.
>     *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEYnjj6z03s4-4iiijjQkNLtN1PCQMyRXnETzX8yYtaEnnYvrqh_cgnI0urrN4ePi9ZgJeVZQKm0n-SzABbCERCjQ9SP-pshVPo4TNrHllpQiLvtV1KjPNi5dQjVZn4zezAw1LieHxgCxW7ZwQIx3PIeZUM)
> 4.  **Desenvolvedor Full Stack Júnior - Nexdom Healthtech**
> 
>     *   Modelo: Remoto
>     *   Requisitos: Desenvolver soluções e realizar manutenção das soluções e sistemas existentes, entregar código limpo e fácil de se manter, priorizando segurança e estabilidade.
>     *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE1qMHulYDAkSSno6X-_GJGeVnrTJTfOVpynzjPdbwivYxBnhdRcPqWucZVgJ_-yQQOtypV75cNG1V5Uced1GPdPzp8GIq41gy55oW0OiepG2XpbsFynZKoxlL3krfAyE9yu2aWF6CVBZOTPR1FwBVxb6EIA7EZrk0nHP5c82CCNGqqZfs=)
> 5.  **DESENVOLVEDOR FULL STACK JUNIOR - Grupo Roma Brasil**
> 
>     *   Local de trabalho: Belo Horizonte - MG
>     *   Modelo de trabalho: Presencial
>     *   Requisitos e qualificações: Ensino Superior completo ou cursando em, Ciências da Computação ou Sistemas de informação, experiência em desenvolvimento de Software sob medida, conhecimentos sólidos em OOP, MVC e Integrações Rest e conhecimentos sólidos na plataforma Microsoft Azure.
>     *   [Link para a vaga](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXESYly3rhLUP4dqPiWfN62e7kxGCnXLyCj4gy60qH0Hufrz-n9UsC5V3oo-m15MX_Zl9VU52DIVDzNP33ey_WuM2mmGbzN3x7W8mXPWV253Rl8TPLzPxblLXnu-mmk1UsySO3vKdFyUH0ycc8pb9g8uk7PPqCh6M7Yeff74zoDvk9p3dFTFxWE=)
> 
> 


--------------------------------------------------------------

🔄 Processando resultados de planejamento...

 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 



> Com prazer! Aqui estão algumas dicas para você se destacar como programador Full Stack Júnior, com foco nas vagas que você mencionou:
> 
> **1. Habilidades e Experiências a Enfatizar:**
> 
> *   **React e Node.js:** Se você tem experiência com React e Node.js, destaque isso, especialmente para a vaga da FIESC. Mesmo que a vaga não especifique, essas tecnologias são muito valorizadas no mercado.
> *   **Conhecimentos em Nuvem (Azure):** Para a vaga do Grupo Roma Brasil, ter conhecimentos em Microsoft Azure é um grande diferencial. Se você tiver projetos ou cursos relacionados, mencione-os.
> *   **OOP, MVC e APIs REST:** A vaga do Grupo Roma Brasil também enfatiza a importância de conhecimentos sólidos em Programação Orientada a Objetos (OOP), Model-View-Controller (MVC) e APIs REST. Certifique-se de que seu currículo e portfólio reflitam sua experiência nessas áreas.
> *   **Docker:** A vaga da FIESC menciona Docker. Se você tiver experiência com contêineres, destaque isso.
> *   **Código Limpo e Segurança:** A vaga da Nexdom Healthtech valoriza a entrega de código limpo e a priorização da segurança. Mostre que você se preocupa com a qualidade do código e a segurança das aplicações.
> 
> **2. Dicas de Organização para Candidaturas:**
> 
> *   **Adapte seu currículo:** Para cada vaga, revise seu currículo e destaque as habilidades e experiências mais relevantes para aquela posição. Use as palavras-chave presentes na descrição da vaga.
> *   **Prepare um portfólio:** Crie um portfólio online com seus projetos. Inclua uma descrição detalhada de cada projeto, as tecnologias utilizadas e o seu papel no desenvolvimento.
> 
> **3. Dicas Gerais para se Destacar no Mercado de Trabalho como Programador Full Stack Júnior:**
> 
> *   **Mantenha-se atualizado:** A área de tecnologia está em constante evolução. Dedique tempo para aprender novas tecnologias e frameworks.
> *   **Contribua com a comunidade:** Participe de fóruns, grupos de discussão e eventos da área. Compartilhe seu conhecimento e aprenda com os outros.
> *   **Desenvolva suas habilidades de comunicação:** A capacidade de se comunicar de forma clara e eficaz é essencial para trabalhar em equipe e apresentar suas ideias.
> *   **Construa um networking:** Conecte-se com outros profissionais da área, participe de eventos e utilize o LinkedIn para expandir sua rede de contatos.
> *   **Aperfeiçoe o inglês:** Muitas empresas e projetos utilizam o inglês como língua de trabalho. Dominar o idioma pode abrir muitas portas.
> 
> Espero que essas dicas ajudem você a se destacar e conquistar a vaga desejada!


--------------------------------------------------------------

 Ficou com alguma dúvida? Pode contar comigo!😉 (caso não tenha dúvidas digite 'N') 

Como me destacar no mercado?

🔄 Processando resposta...


NameError: name 'data_de_hoje' is not defined